[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gretelai/fun-with-synthetic-data/blob/main/Gretel_Bartender.ipynb)

# Lyric Generator
Ever wonder what a Taylor Swift song about corn in the style of Post Malone would sound like? Neither have we. But, with this Synthetic Lyric Generator you
can now answer all the questions that should have never been asked. Lyric Generator uses Gretel GPT, a GPT-3 based model to generate exciting new lyrics in the style of your favorite artists! While we can't guarantee they'll all be
good, they will all be created through the power of AI and Synthetic Data.

You can run the following notebook below to generate your song lyrics. You can sign up for a free Gretel account today to try
this out at https://console.gretel.cloud 



## Generating Synthetic Text

This notebook will walk you through generating synthetic natural language text, similar to the data that you provide to it. This is accomplished by fine-tuning a large scale language model that has been pre-trained on billions of documents and is therefore capable of introducing realistic new variations into the data.
 
To run this notebook, you will need an [API key](https://docs.gretel.ai/environment-setup) from the [Gretel Console](https://console.gretel.ai).

** **Limitations and Biases** **
Large-scale language models such as Gretel GPT may produce untrue and/or offensive content without warning. We recommend having a human curate or filter the outputs before releasing them, both to censor undesirable content and to improve the quality of the results. For more information and examples please see [OpenAI](https://huggingface.co/gpt2#limitations-and-bias) and [EleutherAI](https://huggingface.co/EleutherAI/gpt-neo-125M#limitations-and-biases)'s docs for more details.

## Configure the project
* Install dependencies
* Import libraries
* Log into Gretel and set up a project

In [ ]:
%%capture
!pip install -U gretel-client

In [ ]:
import json

import pandas as pd
from gretel_client import configure_session
from gretel_client.helpers import poll
from gretel_client.projects import create_or_get_unique_project, get_project

DATASET_PATH = 'https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/clean_lyrics.csv'
SEPARATOR = ','

pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Log into Gretel

configure_session(api_key="prompt", cache="yes", endpoint="https://api.gretel.cloud", validate=True, clear=True) #clear=True

project = create_or_get_unique_project(name="lyric-generator")

Gretel Api Key··········
Caching Gretel config to disk.
Using endpoint https://api.gretel.cloud
Logged in as mason@gretel.ai ✅


## Create the model configuration

In this notebook we will use GPT-Neo, a transformer model designed using EleutherAI's replication of OpenAI's GPT-3 Architecture. This model has been pre-trained on the Pile, a large-scale dataset using 300 billion tokens over 572,300 steps. In this introductory example, we will fine-tune GPT-Neo to generate synthetic (and hopefully entertaining) cocktail recipes by fine-tuning across a dataset of well known cocktail recipes. 

In [ ]:
config = {
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 1,
        "epochs": 1,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 0.00005
      }
    }
  ]
}

print(json.dumps(config, indent=2))

{
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 1,
        "epochs": 1,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 5e-05
      }
    }
  ]
}


# Load and preview the training dataset
Create single-column CSV training set by combining intent + SEPARATOR + text.

Gretel GPT only supports synthesizing on one column.

In [ ]:
def create_dataset(dataset_path: str) -> pd.DataFrame:
    """
    Combine intents and text into a single string to pass to GPT-X.
    """
    
    df = pd.read_csv(dataset_path)
    df['combined'] = df['Artist'] + SEPARATOR + df['Title'] + SEPARATOR + df['Clean Lyric'].apply(lambda x: str(x)[:256])
    df = df.dropna()
    return df

df = create_dataset(DATASET_PATH)
df[['combined']].to_csv('finetune.csv', index=False)
df

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric,Clean Lyric,combined
0,0.0,Dua Lipa,New Rules,Dua Lipa,2017.0,2017-06-02,one one one one one talkin' in my sleep at night makin' myself crazy out of my mind out of my mind wrote it down and read it out hopin' it would save me too many times too many times refrain my love he makes me feel like nobody else nobody else but my love he doesn't love me so i tell myself i tell myself pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself i keep pushin' forwards but he keeps pullin' me backwards nowhere to turn no way nowhere to turn no now i'm standin' back from it i finally see the pattern i never learn i never learn refrain but my love he doesn't love me so i tell myself i tell myself i do i do i do pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself practice makes perfect i'm still tryna learn it by heart i got new rules i count 'em eat sleep and breathe it rehearse and repeat it 'cause i i got new pre one don't pick up the phone yeah you know he's only callin' 'cause he's drunk and alone alone two don't let him in uhooh you'll have to kick him out again again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em whoaooh whoaooh whoa i gotta tell them to myself i got new rules i count 'em baby you know i count 'em i gotta tell them to myself don't let him in don't let him in don't don't don't don't don't be his friend don't be his friend don't don't don't don't don't let him in don't let him in don't don't don't don't don't be his friend don't be his friend don't don't don't don't you're gettin' over him,one one one one one talkin' in my sleep at night makin' myself crazy out of my mind out of my mind wrote it down and read it out hopin' it would save me too many times too many times refrain my love he makes me feel like nobody else nobody else but my love he doesn't love me so i tell myself i tell myself pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself i keep pushin' forwards but he keeps pullin' me backwards nowhere to turn no way nowhere to turn no now i'm standin' back from it i finally see the pattern i never learn i never learn refrain but my love he doesn't love me so i tell myself i tell myself i do i do i do pre one don't pick up the phone you know he's only callin' 'cause he's drunk and alone two don't let him in you'll have to kick him out again three don't be his friend you know you're gonna wake up in his bed in the morning and if you're under him you ain't gettin' over him i got new rules i count 'em i got new rules i count 'em i gotta tell them to myself i got new rules i count 'em i gotta tell them to myself practice makes perfect i'm still tryna learn it by heart i got new rules i count 'em eat sleep and breathe it rehearse and repeat it 'cause i i got new pre one don

## Train the synthetic model
In this step, we will task the worker running in the Gretel cloud, or locally, to fine-tune the GPT language model on the source dataset.

In [ ]:
%%time 

model = project.create_model_obj(model_config=config)
model.data_source = "finetune.csv"
model.name = "lyric-generator"
model.submit_cloud()

poll(model)

INFO: Starting poller


{
    "uid": "62cf2393152cbb2acd5fc692",
    "guid": "model_2Bu3dBSTABrLtNfWgOBjqtx6fuA",
    "model_name": "lyric-generator",
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "62cf21ff2901ead70dc2ff4f",
    "project_guid": "proj_2Bu2oP3NCFQJGF1wpEpqcFeYNal",
    "status_history": {
        "created": "2022-07-13T19:57:07.576237Z"
    },
    "last_modified": "2022-07-13T19:57:07.830787Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:fd309f36f570536ef926c49b7ae0a7584359863fc8fb6378dd8d480df8c1b7b9",
    "model_type": "gpt_x",
    "config": {
        "schema_version": "1.0",
        "name": "lyric-generator

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2022-07-13T19:57:25.657754Z  Training gpt_x!
2022-07-13T19:59:56.523322Z  Training in progress
{
    "loss": 3.738,
    "learning_rate": 2.5489247641674596e-09,
    "epoch": 1.0
}
2022-07-13T20:00:06.849350Z  Training in progress
{
    "eval_loss": 3.303887367248535,
    "eval_runtime": 10.3228,
    "eval_samples_per_second": 62.096,
    "eval_steps_per_second": 3.972,
    "epoch": 1.0
}
2022-07-13T20:00:09.712120Z  Training in progress
{
    "train_runtime": 143.4495,
    "train_samples_per_second": 17.888,
    "train_steps_per_second": 2.231,
    "train_loss": 3.7380226135253904,
    "epoch": 1.0
}
2022-07-13T20:00:09.717510Z  Training is completed!
2022-07-13T20:00:09.718011Z  Saving model
2022-07-13T20:00:10.649230Z  Model has been created successfully
2022-07-13T20:01:00

CPU times: user 1.29 s, sys: 203 ms, total: 1.5 s
Wall time: 4min 7s


## Generate synthetic text data
You can now use the fine-tuned synthetic model to generate as much synthetic data as you like. The next cells walk through three ways to generate data.

1.  Generate text records from the model
2.  Generate text records using a single text seed (or prompt)
3.  Generate text records using a unique seed per record

In [ ]:
# Example 1: Generate text records from the model.
pd.set_option('display.max_rows', 20)

record_handler = model.create_record_handler_obj(
    params={"num_records": 20, "maximum_text_length": 128}
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

INFO: Starting poller


{
    "uid": "62cf24d55a03d14992da306c",
    "guid": "model_run_2Bu4HcuRyp0wJ1Fed4QGdskGHjY",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "62cf21ff2901ead70dc2ff4f",
    "project_guid": "proj_2Bu2oP3NCFQJGF1wpEpqcFeYNal",
    "status_history": {
        "created": "2022-07-13T20:02:29.233000Z"
    },
    "last_modified": "2022-07-13T20:02:29.412000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:fd309f36f570536ef926c49b7ae0a7584359863fc8fb6378dd8d480df8c1b7b9",
    "model_id": "62cf2393152cbb2acd5fc692",
    "model_guid": "model_2Bu3dBSTABrLtNfWgOBjqtx6fuA",
    "action": "gpt

INFO: Status is created. A job has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-07-13T20:03:03.937693Z  Sampling 20 records conditioned on ''.
2022-07-13T20:03:03.939604Z  Using device 'cpu'
2022-07-13T20:03:05.838386Z  Generating 20 records
2022-07-13T20:03:14.603893Z  [2/20] records complete.
{
    "completed_count": 2,
    "total": 20
}
2022-07-13T20:03:23.390944Z  [4/20] records complete.
{
    "completed_count": 4,
    "total": 20
}
2022-07-13T20:03:32.160267Z  [6/20] records complete.
{
    "completed_count": 6,
    "total": 20
}
2022-07-13T20:03:40.859814Z  [8/20] records complete.
{
    "completed_count": 8,
    "total": 20
}
2022-07-13T20:03:49.536317Z  [10/20] records complete.
{
    "completed_count": 10,
    "total": 20
}
2022-07-13T20:03:58.033006Z  [12/20] records complete.
{
    "completed_count": 12,
    "total": 20
}
2022-07-13T20:04:06.521158Z  [14/20] records complete.
{
    "complet

,combined
0,i told myself i'd never lose the day of my life and the world would tell you the truth and my life would be all wrong for me so i wouldn't give it to you but i would not take it back for you and you the one i'd never get back to before i've given you everything and i'm waiting for your answer i'm waiting for your answer i'm a sucker for the best and the worst but there's nothing you could i'd never have let the last moment end and i'd never have let that happen on this earth if i
1,if (bool) bazb (xmin=min (bazb min (ymin ymin zmin min (bazb min ymin zmin min (ymin min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min min
2,d. (salesperson) i never thought it would be this way without you do you want to be your friend you won't give it back to me oh yeah yes yes we'll get over it yeah yeah we'll have it up and running yeah i do not think it's a deal we'll never see again yeah it's not my fault at all do you want to be my girl you will have everything you ever want for me if you want to be my friend you will have everything you ever want for me if you want to be my girl do you want to be my
3,"i would have been happy if it's what you do to me i've tried to be more than i ever wanted to be pre, i want the things that you do to me i have tried to be more than i ever wanted to be pre if you're ever in the dark you should talk more than once a day you should be the one to put you in my shadow and take me out of my shadow if you ever come here i can be the one to talk more than once a day you should be the one to put you in my shadow and take me out of my shadow pre oh and if"
4,*
5,pinch your face the way you say you're the only one i've ever called so far i can't take it anymore i'm not coming back or what's wrong when i say what i am feel your arms reach your head and we kiss as we make our way to the door and you say i'm here for you it's not something i'm for anymore my mind never stopped searching for the right way to find it and i could be you forever it's the best i can do now if you know i am there to take me away i am the one who's left to get
6,viral/santa dna nada yo vidiados so many people just go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies of you go and watch the ***** movies
7,i say 'cause i used to be like this or like this you'd have a lot of work to do if i had this i'd go to the hospital and take me to the park where i'd like to be so i could stay in the park yeah you could be the best of all worlds pre i guess you did all the work but it's a bad boy i bet you did it all the way oh oh i guess you did it all the way to the grave of the devil oh i guess you did it all the way to the grave of the
8,"i'm a fan of your music, you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your music you're my idol i'm a fan of your"
9,NaN


In [ ]:
# Example 2: Generate text with an optional "prompt" which is used to condition
# model generation.

record_handler = model.create_record_handler_obj(
    params={"num_records": 5, 
            "maximum_text_length": 128,
            "prompt": "Taylor Swift,Happy Ducks,Bilbo was a happy duck who"}
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

INFO: Starting poller


{
    "uid": "62cf26365a03d14992da306d",
    "guid": "model_run_2Bu504qkL7p6Iv2vFLPD0Zs0NAL",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "62cf21ff2901ead70dc2ff4f",
    "project_guid": "proj_2Bu2oP3NCFQJGF1wpEpqcFeYNal",
    "status_history": {
        "created": "2022-07-13T20:08:22.827000Z"
    },
    "last_modified": "2022-07-13T20:08:22.985000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:fd309f36f570536ef926c49b7ae0a7584359863fc8fb6378dd8d480df8c1b7b9",
    "model_id": "62cf2393152cbb2acd5fc692",
    "model_guid": "model_2Bu3dBSTABrLtNfWgOBjqtx6fuA",
    "action": "gpt

INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-07-13T20:08:54.417166Z  Sampling 5 records conditioned on 'Taylor Swift,Happy Ducks,Bilbo was a happy duck who'.
2022-07-13T20:08:54.418909Z  Using device 'cpu'
2022-07-13T20:08:56.279890Z  Generating 5 records
2022-07-13T20:09:17.560330Z  Successfully generated 5 records
2022-07-13T20:09:17.594050Z  Uploading artifacts to Gretel Cloud


,combined
0,got a chance to shine in the sunlight so happy that they can be happy that you came to visit with the birds in the garden and that it's been easy to spend too much time with birds that you know your way around and that you have been lonely because we are happy and lonely and you are lonely and lonely and when you're sad you're not a happy duck who gets a chance to shine in the sunlight so happy that they can be happy that they can be happy that they can be happy that they can be happy that they can be happy that they can be happy that they can be happy that they can be happy that they can
1,fell into the hands of a lonely soul when the sun came down from the clouds was it a birthday or was it the end of the world when my love was at home i cried so much that the tears turned into tears at the thought of you i just wanted my soul to make sure your sweet angel was right there with me all alone so i put the headphones to my ears so i didn't have to scream my heart was heavy so when i got the message that she was hurt it took a little while and i came down the stairs hoping she had left me alone we couldn't talk so i took a taxi to get home my face was
2,liked ducks so much he liked his blue bluish eyes the first time i saw your face when i was in your arms we met a lot of ducks it was always the same every time you go back home there's no way we can survive all that ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks are born to you all your ducks
3,could talk about love like a duck who could kiss a happy duck with the kiss in a million years my world was so different than you and i were so different than you and i were so different than you and i was so different than you and i was so different than you and i was so different than you and i you're the reason that i'm here and you're the reason that i'm here happy ducks are the real reason that i'm here happy ducks who's this world i'm living in are the real reason that i'm living in are the real reason that i'm living
4,was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it was happy that it


In [ ]:
# Example 3: Generate text with optional "prompts" to condition model generation 
# with an individual prompt for each record in CSV format.

ARTISTS = ("Taylor Swift", "Lady Gaga", "Post Malone")
SONG_THEMES = ('dogs', 'corn', 'bicycles', 'sharks', 'hot dogs')

prompts = pd.DataFrame([f'{artist}{SEPARATOR}{theme}{SEPARATOR}' 
                        for theme in SONG_THEMES for artist in ARTISTS], columns=["text"])
prompts.to_csv('prompts.csv', index=False)

record_handler = model.create_record_handler_obj(
    params={"maximum_text_length": 128},
    data_source='prompts.csv'
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

INFO: Starting poller


{
    "uid": "62cdd04d07326a4a49ab1ee1",
    "guid": "model_run_2BrDaRXJWDg5g6yIwLYJv58UByg",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "62cd9720e1402b05c90b61ad",
    "project_guid": "proj_2Bqjv8HC6ze8lsYhbeprSaXBIKL",
    "status_history": {
        "created": "2022-07-12T19:49:33.091000Z"
    },
    "last_modified": "2022-07-12T19:49:33.269000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:d3cdb5e6cf17f3e2e3312bf81a4750e3e03a7e4bdecdeab3e7b5b1081a9ff82b",
    "model_id": "62cdc48f88c5736746b1a075",
    "model_guid": "model_2Br7UnfYYmg2qUhvyJ7DyhoyTeK",
    "action": "gpt

INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-07-12T19:49:56.160905Z  Sampling 15 records using conditioning input...
2022-07-12T19:49:56.162345Z  Using device 'cpu'
2022-07-12T19:49:58.033504Z  Generating 15 records
2022-07-12T19:50:02.303015Z  [1/15] records complete.
{
    "completed_count": 1,
    "total": 15
}
2022-07-12T19:50:06.471368Z  [2/15] records complete.
{
    "completed_count": 2,
    "total": 15
}
2022-07-12T19:50:10.727180Z  [3/15] records complete.
{
    "completed_count": 3,
    "total": 15
}
2022-07-12T19:50:14.953774Z  [4/15] records complete.
{
    "completed_count": 4,
    "total": 15
}
2022-07-12T19:50:19.184892Z  [5/15] records complete.
{
    "completed_count": 5,
    "total": 15
}
2022-07-12T19:50:23.388726Z  [6/15] records complete.
{
    "completed_count": 6,
    "total": 15
}
2022-07-12T19:50:27.562515Z  [7/15] records complete.
{
    "completed_count": 7,
    "total": 15
}
2022-07-12T1

,combined
0,no matter how it shakes down and shakes down again and again get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and get up and
1,"do they make me feel like a cat with a hole in my chest, pre the game is still alive i will go and play i will leave you and your cats in peace if you're not your best friend are you still alive pre when i think about how my own life goes if the thing you have made me feel like i could never leave you i know it is not a big deal pre i would hate to think that you've been hurt so hard the pain would stay deep but it would leave you alone when it's all over i would feel like you've been mine what you do"
2,and i'm trying to forget about all the things i don't wanna forget and if i forget they're gone then it's time to run back to the place i came back to yesterday just yesterday and we've been fighting this whole thing so far we can keep fighting so we don't get far when we come back for the rest of the season maybe tonight we'll be okay we'll make a show of ourselves all the way to the end so i'm gonna run the dogs around every corner that way so i'm gonna walk out the door right here and there that's what i'm trying to forget we don't wanna get far now it
3,oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh oh
4,corn' they say we gotta eat these things when we want 'em' they say we must be in 'em' they say we gotta go out and eat 'em when we want 'em they say we gotta go out and eat 'em when we want 'em when we want 'em 'bout the world they say we gotta go out and eat 'em when we wanna be 'bout the world they say we gotta go out and eat 'em when we wanna be 'bout the world they say we gotta go out and eat 'em when we wanna be 'bout the world they say we gotta go out and
5,i am on vacation but i miss my job i miss my friends' homes oh and i miss the love of my life oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends' homes oh and i miss my friends'
6,i love my bike that's so hot it's ready to go but now my mind has been locked up pre and now i'm locked up and that's the best part the car just so loud it's too loud and i can't get out it's too early for me but i'm ready to go and i'm ready to go and i'm ready to go but now my mind is locked up and i'm locked up and that's the worst part pre and now i'm locked up and that's the worst part and i'm locked up and that's the worst part and i'm locked up and that's the worst part
7,you want a car with a few thousand miles on it with my wife and a little girl on the beach you drive a car with a few thousand miles on it with my wife and a little girl on the beach and the girls play in a club i'm a virgin it's the coolest ****** on the planet we were in love with each other but we were never born we were not born we were never born i got me a fake one i love you and it's your *** ******* just like my *** ******* i love you and i've been living through my sorrows i just wanna know that the world will end my
8,we're in trouble right now don't get too far we'll ju